# **CATEGORÍA FyV predictions**

## **Importar librerías**

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
import holidays

In [3]:
# facebook prophet
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

## **Cargar datos**

In [4]:
prod_fecha_fyv = pd.read_csv('../../data/sql/venta/prod_fecha_fyv.csv')

ventas_productos = pd.read_csv('../../data/ventas_productos.csv')

In [5]:
prod_fecha_fyv.info()

,fecha,dia_semana,categoria,producto,formato,venta
0,2023-01-02,lunes,Fruta y verdura,Plátano de Canarias,Pieza 160 g aprox.,17
1,2023-01-02,lunes,Fruta y verdura,Banana,Pieza 170 g aprox.,23
2,2023-01-02,lunes,Fruta y verdura,Plátano macho,Pieza 340 g aprox.,35
3,2023-01-02,lunes,Fruta y verdura,Uva blanca sin semillas,Bandeja 500 g aprox.,12
4,2023-01-02,lunes,Fruta y verdura,Uva roja sin semillas,Bandeja 500 g aprox.,1
...,...,...,...,...,...,...
26420,2023-06-30,viernes,Fruta y verdura,Mix de verduras paisana para micro,Paquete 300 g,0
26421,2023-06-30,viernes,Fruta y verdura,Sopa juliana para micro,Paquete 400 g,0
26422,2023-06-30,viernes,Fruta y verdura,Patata para micro,Paquete 400 g,0
26423,2023-06-30,viernes,Fruta y verdura,Batata para micro,Paquete 400 g,0


In [6]:
ventas_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26425 entries, 0 to 26424
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   fecha       26425 non-null  object
 1   dia_semana  26425 non-null  object
 2   categoria   26425 non-null  object
 3   producto    26425 non-null  object
 4   formato     26425 non-null  object
 5   venta       26425 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


## **Transformar datos**

In [7]:
# cambiar columna "fecha" sea tipo "datetime"
prod_fecha_fyv['fecha'] = pd.to_datetime(prod_fecha_fyv['fecha'])

prod_fecha_fyv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26425 entries, 0 to 26424
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   fecha       26425 non-null  datetime64[ns]
 1   dia_semana  26425 non-null  object        
 2   categoria   26425 non-null  object        
 3   producto    26425 non-null  object        
 4   formato     26425 non-null  object        
 5   venta       26425 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.2+ MB


In [8]:
# definir rango de fechas
fecha_inicio = pd.to_datetime('2023-06-01')
fecha_fin = pd.to_datetime('2023-06-30')

# eliminar filas dentro de rango de fechas
prod_fecha_fyv = prod_fecha_fyv[~((prod_fecha_fyv['fecha'] >= fecha_inicio) & (prod_fecha_fyv['fecha'] <= fecha_fin))]

prod_fecha_fyv

,fecha,dia_semana,categoria,producto,formato,venta
0,2023-01-02,lunes,Fruta y verdura,Plátano de Canarias,Pieza 160 g aprox.,17
1,2023-01-02,lunes,Fruta y verdura,Banana,Pieza 170 g aprox.,23
2,2023-01-02,lunes,Fruta y verdura,Plátano macho,Pieza 340 g aprox.,35
3,2023-01-02,lunes,Fruta y verdura,Uva blanca sin semillas,Bandeja 500 g aprox.,12
4,2023-01-02,lunes,Fruta y verdura,Uva roja sin semillas,Bandeja 500 g aprox.,1
...,...,...,...,...,...,...
21870,2023-05-31,miércoles,Fruta y verdura,Mix de verduras paisana para micro,Paquete 300 g,31
21871,2023-05-31,miércoles,Fruta y verdura,Sopa juliana para micro,Paquete 400 g,18
21872,2023-05-31,miércoles,Fruta y verdura,Patata para micro,Paquete 400 g,35
21873,2023-05-31,miércoles,Fruta y verdura,Batata para micro,Paquete 400 g,30


In [9]:
# generar nueva columna con nombre único para cada combinación de producto y formato
prod_fecha_fyv['producto_formato'] = prod_fecha_fyv['producto'] + '_' + prod_fecha_fyv['formato']

# obtener lista única de nombres de producto en "prod_fecha_fyv"
nombres_productos = prod_fecha_fyv['producto_formato'].unique()

# generar diccionario para almacenar dataframes separados por "producto_formato"
dfs_productos = {}

# recorrer lista "nombres_productos" y filtrar "prod_fecha_fyv" por cada nombre
for producto_formato in nombres_productos:
    df_producto = prod_fecha_fyv[prod_fecha_fyv['producto_formato'] == producto_formato].copy()

     # reiniciar índice subdataframe
    df_producto.reset_index(drop=True, inplace=True)

    dfs_productos[producto_formato] = df_producto

## **Predicciones**

### **Prophet**

In [10]:
# realizar función para generar predicciones de venta futuras para cada producto
def preds_prophet(subdf_producto):
    # crear objeto de Prophet
    m = Prophet()

    # renombrar columnas requeridas para aplicar Prophet ("ds" para columna "fecha", "y" para columna de "venta")
    df_prophet = subdf_producto[['fecha', 'venta']].rename(columns={'fecha' : 'ds', 'venta' : 'y'})

    # ajustar modelo Prophet
    m.fit(df_prophet)

    # generar predicciones para 30 días adicionales
    futuras_fechas = m.make_future_dataframe(periods=30)
    predicciones = m.predict(futuras_fechas)

    return predicciones

In [11]:
# generar diccionario para almacenar dataframes con predicciones separados por "producto_formato"
preds_productos = {}

# recorrer lista "nombres_productos" y filtrar "prod_fecha_fyv" por cada nombre
for producto_formato, subdf_producto in dfs_productos.items():
    predicciones = preds_prophet(subdf_producto)
    preds_productos[producto_formato] = predicciones

00:23:40 - cmdstanpy - INFO - Chain [1] start processing
00:23:41 - cmdstanpy - INFO - Chain [1] done processing
00:23:41 - cmdstanpy - INFO - Chain [1] start processing
00:23:41 - cmdstanpy - INFO - Chain [1] done processing
00:23:41 - cmdstanpy - INFO - Chain [1] start processing
00:23:41 - cmdstanpy - INFO - Chain [1] done processing
00:23:41 - cmdstanpy - INFO - Chain [1] start processing
00:23:41 - cmdstanpy - INFO - Chain [1] done processing
00:23:41 - cmdstanpy - INFO - Chain [1] start processing
00:23:41 - cmdstanpy - INFO - Chain [1] done processing
00:23:41 - cmdstanpy - INFO - Chain [1] start processing
00:23:41 - cmdstanpy - INFO - Chain [1] done processing
00:23:41 - cmdstanpy - INFO - Chain [1] start processing
00:23:41 - cmdstanpy - INFO - Chain [1] done processing
00:23:41 - cmdstanpy - INFO - Chain [1] start processing
00:23:41 - cmdstanpy - INFO - Chain [1] done processing
00:23:41 - cmdstanpy - INFO - Chain [1] start processing
00:23:41 - cmdstanpy - INFO - Chain [1]

In [12]:
cols_objetivo = ["ds", "yhat"]
preds_final = {}

for producto_formato, predicciones in preds_productos.items():
    predicciones_seleccionadas = predicciones[cols_objetivo].copy()
    predicciones_seleccionadas = predicciones_seleccionadas.rename(columns={"ds": "fecha", "yhat": "prevision"})
    predicciones_seleccionadas["prevision"] = predicciones_seleccionadas["prevision"].astype(int)
    preds_final[producto_formato] = predicciones_seleccionadas


In [13]:
# unir subdataframes en un dataframe global
preds_prophet_fyv = pd.concat(list(preds_final.values()), ignore_index=True)

preds_prophet_fyv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27125 entries, 0 to 27124
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   fecha      27125 non-null  datetime64[ns]
 1   prevision  27125 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 424.0 KB


In [14]:
# obtener festivos nacionales en España para el año 2023
festivos_espana = holidays.Spain(years=2023)

# eliminar filas que sean domingos en la columna "fecha"
preds_prophet_fyv = preds_prophet_fyv[~(preds_prophet_fyv['fecha'].dt.dayofweek == 6)]

# eliminar filas correspondientes a festivos nacionales en España
preds_prophet_fyv = preds_prophet_fyv[~preds_prophet_fyv['fecha'].isin(festivos_espana)]

# reiniciar índice del dataframe
preds_prophet_fyv.reset_index(drop=True, inplace=True)

preds_prophet_fyv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26425 entries, 0 to 26424
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   fecha      26425 non-null  datetime64[ns]
 1   prevision  26425 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 413.0 KB


In [21]:
# añadir predicciones a columna "prevision"
ventas_productos['prevision'] = preds_prophet_fyv['prevision']

ventas_productos

,venta_id,producto_id,venta,prevision
0,1,1,17,19
1,2,1,11,13
2,3,1,5,15
3,4,1,8,15
4,5,1,56,35
...,...,...,...,...
26420,147,175,0,25
26421,148,175,0,17
26422,149,175,0,22
26423,150,175,0,21


In [20]:
ventas_productos

,venta_id,producto_id,venta,prevision
0,1,1,17,19
1,2,1,11,13
2,3,1,5,15
3,4,1,8,15
4,5,1,56,35
...,...,...,...,...
26420,147,175,0,25
26421,148,175,0,17
26422,149,175,0,22
26423,150,175,0,21


In [14]:
# visualizar predicciones
fyv_plot_1 = plot_plotly(m, preds_prophet_fyv)

fyv_plot_1

In [15]:
fyv_plot_2 = plot_components_plotly(m, preds_prophet_fyv)

fyv_plot_2

## **Exportar predicciones**

In [22]:
# exportar predicciones transformadas a ".csv"
ventas_productos.to_csv('../../data/ventas_productos.csv', index=False)